In [2]:
%pip install numpy pandas scikit-learn xgboost lightgbm requests tqdm joblib

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install nba_api pandas scikit-learn numpy


Note: you may need to restart the kernel to use updated packages.


In [9]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
games = leaguegamefinder.LeagueGameFinder(season_nullable="2024-25", league_id_nullable="00").get_data_frames()[0]
games.head()


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42024,1610612760,OKC,Oklahoma City Thunder,0042400221,2025-05-05,OKC vs. DEN,L,241,119,...,0.714,13,30,43,27,12,11,9,27,-2.0
1,42024,1610612738,BOS,Boston Celtics,0042400211,2025-05-05,BOS vs. NYK,L,264,105,...,0.846,19,40,59,18,4,5,13,22,-3.0
2,42024,1610612752,NYK,New York Knicks,0042400211,2025-05-05,NYK @ BOS,W,265,108,...,0.548,12,37,49,19,10,2,12,19,3.0
3,42024,1610612743,DEN,Denver Nuggets,0042400221,2025-05-05,DEN @ OKC,W,240,121,...,0.818,21,42,63,18,6,5,18,21,2.0
4,42024,1610612744,GSW,Golden State Warriors,0042400157,2025-05-04,GSW @ HOU,W,240,103,...,0.778,3,35,38,24,6,8,7,14,14.0


In [16]:
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games = games[games['MIN'] > 0]
games = games[~games['MATCHUP'].str.contains('All-Star', na=False)]

playoff_teams = [
    1610612737, 1610612738, 1610612751, 1610612760,
    1610612744, 1610612743, 1610612764, 1610612752,
    1610612745, 1610612763, 1610612762, 1610612740,
    1610612749, 1610612753, 1610612766, 1610612755
]
games = games[games['TEAM_ID'].isin(playoff_teams)]

games['a_home'] = games['MATCHUP'].str.contains(r' vs\.').astype(int)

a = games.rename(columns=lambda c: 'a_'+c if c not in ['GAME_ID','GAME_DATE'] else c)
b = games.rename(columns=lambda c: 'b_'+c if c not in ['GAME_ID','GAME_DATE'] else c)

merged = (
    a.merge(b, on=['GAME_ID','GAME_DATE'])
     .query('a_TEAM_ID != b_TEAM_ID')
)

merged = merged[merged['a_a_home'] == 1].reset_index(drop=True)

merged['diff_pts'] = merged['a_PTS'] - merged['b_PTS']
merged['a_win']   = (merged['diff_pts'] > 0).astype(int)

X = merged[['diff_pts','a_a_home']]
y = merged['a_win']

merged.head()







,a_SEASON_ID,a_TEAM_ID,a_TEAM_ABBREVIATION,a_TEAM_NAME,GAME_ID,GAME_DATE,a_MATCHUP,a_WL,a_MIN,a_PTS,...,b_REB,b_AST,b_STL,b_BLK,b_TOV,b_PF,b_PLUS_MINUS,b_a_home,diff_pts,a_win
0,42024,1610612760,OKC,Oklahoma City Thunder,0042400221,2025-05-05,OKC vs. DEN,L,241,119,...,63,18,6,5,18,21,2.0,0,-2,0
1,42024,1610612738,BOS,Boston Celtics,0042400211,2025-05-05,BOS vs. NYK,L,264,105,...,49,19,10,2,12,19,3.0,0,-3,0
2,42024,1610612745,HOU,Houston Rockets,0042400157,2025-05-04,HOU vs. GSW,L,241,89,...,38,24,6,8,7,14,14.0,0,-14,0
3,42024,1610612744,GSW,Golden State Warriors,0042400156,2025-05-02,GSW vs. HOU,L,239,107,...,46,23,12,5,11,18,8.0,0,-8,0
4,42024,1610612745,HOU,Houston Rockets,0042400155,2025-04-30,HOU vs. GSW,W,241,131,...,49,25,10,5,14,27,-15.0,0,15,1


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, shuffle=False)
model = LogisticRegression().fit(X_train, y_train)
preds = model.predict_proba(X_val)[:,1]
print("AUC =", roc_auc_score(y_val, preds))

AUC = 1.0
